In [1]:
from util import get_freq
import matplotlib.pyplot as plt
import numpy as np
from random import uniform
from NPSmethods import readInImages
from NPSmethods import pupilFunc
from numpy.fft import ifftshift, irfft2
import pickle as pkl
import matplotlib.cm as cm
from komm import AWGNChannel
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
#Geranate random assortment of parameters
def make_Paras(seedParas, numToGen):
    outPara = np.empty(shape =(numToGen, len(seedParas)))
    paraTemp = [None] * len(seedParas)
    for i in range(numToGen):
        if i == 0:
            paraTemp = seedParas.copy()
        else:
            for j in range(len(seedParas)):
                para = seedParas[j]
                if j ==3 or j == 5 or j == 6:
                    para = para * uniform(.5, 2)
                paraTemp[j] = para
        outPara[i] = paraTemp
    return outPara
        

#Generate M2k_Fit
def make_M2k_Fit(paras, imgSysData):
    
    A, tau, S0, alpha, phi, beta, delta_s = paras
    _, _, K_X, K_Y = get_freq(imgSysData["CCDPixelSize"],
                                      imgSysData["magnification"], (100,100))
    d = imgSysData["wavelen"] / (2*np.pi*imgSysData["NA"])
    R_p, Theta_p = np.abs(K_X + 1j*K_Y) * d, np.angle(K_X + 1j*K_Y)
    p1 = pupilFunc(R_p, Theta_p + np.pi, tau, S0, alpha, phi, beta)
    p2 = np.conj(pupilFunc(R_p, Theta_p, tau, S0, alpha, phi, beta)) * \
            np.exp(-2*1j*delta_s)
    PSF = (p1 + p2) / (2 * np.cos(delta_s))
    M2k = np.abs(PSF)**2
    M2k_Fit = A * M2k
    M2k_Fit[M2k_Fit.shape[0]//2, M2k_Fit.shape[1]//2] = 0
    
    return M2k_Fit

#Generate NPS noise
def generate_Noise(M2k_Fit, idealOD):
    M2k_Fit = M2k_Fit.flatten()
    NPSavgs = np.zeros(shape = len(M2k_Fit))
    signal_region = np.where(M2k_Fit != 0)
    NSR = np.random.normal(loc = 0.1, scale = 0.2, size = len(signal_region[0]))
    bkg_noise = np.random.normal(loc = np.sqrt(np.max(M2k_Fit)) * .011, scale = 0.0025 * np.sqrt(np.max(M2k_Fit)), size = len(M2k_Fit))
    

    for j in range(len(signal_region[0])):
        NPSavgs[signal_region[0][j]] = M2k_Fit[signal_region[0][j]] + M2k_Fit[signal_region[0][j]] * NSR[j]
    for k in range(len(bkg_noise)):
        NPSavgs[k] += bkg_noise[k]

    NPSavgs = np.reshape(NPSavgs, (100, 100))
    
    return NPSavgs

#Turn one NPS to one OD
def NPSavg_To_Single(NPSavgs, ODAvg, numNPSs):
    NPSs = []
    NPSnum = 0
    
    for NPSavg in NPSavgs:
        NPSnum += 1
        print("NPS number: %d" % NPSnum)
        NPSavg = NPSavg.flatten()
        for i in range(numNPSs):
            NPStemp = NPSavg.copy()
            for j in range(len(NPStemp)):
                NPStemp[j] += NPStemp[j] * (np.random.exponential(scale = 1.5, size = 1) - 1)
            NPStemp = np.reshape(NPStemp, (100, 100))
            NPSs.append(NPStemp)
            
    return NPSs

In [3]:
folderpath = './raw_image/141902/'
savepath = './results/'
seedParas = [0.35060326843236184, 
             0.6484441665068852, 
             -2.63564159087613,
             0.5094606166480791, 
             -1.8773441762223027, 
             0.8206242586655179, 
             1.0257364487180909]

numNPSs = 50
numParas = 1000

NPSavgs = []
fakeNPSs = []
savedParas = []

_, idealOD, _, _, _, _ = readInImages(folderpath, 23, 0, trapRegion = (slice(100, 200), slice(100, 200)), noiseRegion = (slice(0, 300), slice(0, 300)))

imgSysData = { 
    "CCDPixelSize": 13,      # pixel size of the CCD, in micron 
    "magnification": 27,      # 799.943 / 29.9099, # magnification of the imaging system 
    "wavelen"     : 0.852,     # wavelength of the imaging beam, in micron 
    "NA"          : 0.37,      # numerical aperture of the objective 
    "ODtoAtom"    : 13
} 

for paras in make_Paras(seedParas, numParas):
    NPSavgs.append(generate_Noise(make_M2k_Fit(paras, imgSysData), idealOD))
    for j in range(numNPSs):
        savedParas.append(paras)
fakeNPSs = NPSavg_To_Single(NPSavgs, idealOD, numNPSs)

NPS number: 1
NPS number: 2
NPS number: 3
NPS number: 4
NPS number: 5
NPS number: 6
NPS number: 7
NPS number: 8
NPS number: 9
NPS number: 10
NPS number: 11
NPS number: 12
NPS number: 13
NPS number: 14
NPS number: 15
NPS number: 16
NPS number: 17
NPS number: 18
NPS number: 19
NPS number: 20
NPS number: 21
NPS number: 22
NPS number: 23
NPS number: 24
NPS number: 25
NPS number: 26
NPS number: 27
NPS number: 28
NPS number: 29
NPS number: 30
NPS number: 31
NPS number: 32
NPS number: 33
NPS number: 34
NPS number: 35
NPS number: 36
NPS number: 37
NPS number: 38
NPS number: 39
NPS number: 40
NPS number: 41
NPS number: 42
NPS number: 43
NPS number: 44
NPS number: 45
NPS number: 46
NPS number: 47
NPS number: 48
NPS number: 49
NPS number: 50
NPS number: 51
NPS number: 52
NPS number: 53
NPS number: 54
NPS number: 55
NPS number: 56
NPS number: 57
NPS number: 58
NPS number: 59
NPS number: 60
NPS number: 61
NPS number: 62
NPS number: 63
NPS number: 64
NPS number: 65
NPS number: 66
NPS number: 67
NPS 

NPS number: 520
NPS number: 521
NPS number: 522
NPS number: 523
NPS number: 524
NPS number: 525
NPS number: 526
NPS number: 527
NPS number: 528
NPS number: 529
NPS number: 530
NPS number: 531
NPS number: 532
NPS number: 533
NPS number: 534
NPS number: 535
NPS number: 536
NPS number: 537
NPS number: 538
NPS number: 539
NPS number: 540
NPS number: 541
NPS number: 542
NPS number: 543
NPS number: 544
NPS number: 545
NPS number: 546
NPS number: 547
NPS number: 548
NPS number: 549
NPS number: 550
NPS number: 551
NPS number: 552
NPS number: 553
NPS number: 554
NPS number: 555
NPS number: 556
NPS number: 557
NPS number: 558
NPS number: 559
NPS number: 560
NPS number: 561
NPS number: 562
NPS number: 563
NPS number: 564
NPS number: 565
NPS number: 566
NPS number: 567
NPS number: 568
NPS number: 569
NPS number: 570
NPS number: 571
NPS number: 572
NPS number: 573
NPS number: 574
NPS number: 575
NPS number: 576
NPS number: 577
NPS number: 578
NPS number: 579
NPS number: 580
NPS number: 581
NPS numb

In [5]:
K_x, K_y, _, _ = get_freq(imgSysData["CCDPixelSize"], imgSysData["magnification"], (100,100))
print(np.shape(NPSavgs[0]))
for imnum in range(100):
    fig = plt.figure(figsize=(5.5, 4.5))
    ax = fig.add_subplot(111)
    im = ax.pcolor(K_x, K_y, NPSavgs[imnum], cmap=cm.jet, vmin = 0, vmax = NPSavgs[imnum].max())
    plt.colorbar(im)
    fig.savefig("C:\\Users\\eric0\\OneDrive\\Desktop\\Cold Physics\\fake_NPSavg_ims\\fake_NPSavg_" + str(imnum + 1)) 
    plt.close(fig)

(100, 100)


In [6]:
for imnum in range(100):
    fig = plt.figure(figsize=(5.5, 4.5))
    ax = fig.add_subplot(111)
    im = ax.pcolor(K_x, K_y, fakeNPSs[imnum], cmap=cm.jet, vmin = 0, vmax = fakeNPSs[imnum].max())
    plt.colorbar(im)
    fig.savefig("C:\\Users\\eric0\\OneDrive\\Desktop\\Cold Physics\\fake_NPS_ims\\fake_NPS_" + str(imnum + 1))
    plt.close(fig)

In [7]:
pkl.dump(NPSavgs, open(savepath + "fakeNPSavgs", "wb"))
pkl.dump(fakeNPSs, open(savepath + "fakeNPSs", "wb"))
pkl.dump(savedParas, open(savepath + "savedParas", "wb"))

In [8]:
fakeNPSs = np.array(fakeNPSs).reshape((len(fakeNPSs),-1))
savedParas = np.array(savedParas)[:, [3, 5, 6]]
savedParas = np.delete(savedParas, np.where(np.amax(fakeNPSs, axis = 1) > 20), 0)
fakeNPSs = np.delete(fakeNPSs, np.where(np.amax(fakeNPSs, axis = 1) > 20), 0)
scaler = StandardScaler()
yscaler = scaler.fit(savedParas)
pkl.dump(yscaler, open(savepath + "yscaler", "wb"))
savedParas = scaler.transform(np.array(savedParas))
trainin, testin, trainout, testout = train_test_split(fakeNPSs, savedParas, test_size=0.25, shuffle=True)

In [9]:
trainin = tf.convert_to_tensor(trainin)
trainout = tf.convert_to_tensor(trainout)
testin = tf.convert_to_tensor(testin)
testout = tf.convert_to_tensor(testout)

In [10]:
pkl.dump(trainin, open(savepath + "trainin", "wb"))
pkl.dump(trainout, open(savepath + "trainout", "wb"))
pkl.dump(testin, open(savepath + "testin", "wb"))
pkl.dump(testout, open(savepath + "testout", "wb"))